## Generate videos by prompting with ground truth captions
Oracle baseline for BOLDMoments videos

In [2]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import json
import shutil
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
import os
from utils import transform_vids_to_gifs, vid_to_gif, frames_to_vid

%load_ext autoreload
%autoreload 2


## Load captions 

In [3]:
# Load captions
annots = json.load(open('data/annotations.json', 'r')) # captions located in annots.values()[0]['text_descriptions']
print("annots['0001']:",annots['0001'])

annots['0001']: {'bmd_matrixfilename': 'vid_idx0001', 'MiT_url': 'https://data.csail.mit.edu/soundnet/actions3/wetting/0-0-1-6-7-2-8-0-17500167280.mp4', 'MiT_filename': 'wetting/0-0-1-6-7-2-8-0-17500167280.mp4', 'set': 'train', 'objects': ['red-breasted merganser', 'duck', 'American coot', 'goose', 'killer whale'], 'scenes': ['swimming hole', 'natural lake', 'watering hole', 'pond', 'ice floe'], 'actions': ['swimming', 'swimming', 'paddling', 'eating/feeding', 'swimming'], 'text_descriptions': ['A duck is swimming in a lake searching for food', 'A duck is floating atop the blue sparkly looking water.', 'A duck swims along in the water and pecks at the water.', 'A mallard is in the water alone swimming around and putting its beak in.', 'A duck swims in the daytime while pecking at the water.'], 'spoken_transcription': 'in a large mostly still body of water we see a duck swimming and pecking at the surface with his beak', 'memorability_score': 0.8147719988084737, 'memorability_decay': -0

## Functions to generate videos

In [6]:

def generate_videos_from_annots_with_gradio_api(annots, start_from=0):
    from gradio_client import Client

    client = Client("https://fffiloni-zeroscope--x84m2.hf.space/")
    for i, a in annots.items():
        if int(i) < start_from:
            continue
        for c in range(len(a['text_descriptions'])):
            prompt = a['text_descriptions'][c]
            print(prompt)
            result = client.predict(
                            prompt,	# str in 'Prompt' Textbox component
                            api_name="/zrscp"
            )

            # Move video to correct folder
            shutil.move(result, f'./oracle_gens/{i}_captionnumber{c}_{prompt}.mp4')

            # Make gif
            vid_to_gif(f'./oracle_gens/{i}_captionnumber{c}_{prompt}.mp4', f'./oracle_gens/{i}_captionnumber{c}_{prompt}.gif')
            break

def generate_videos_from_annots_with_local(annots, start_from=0, n_samples=5, save_frames=False):
    pipe = DiffusionPipeline.from_pretrained("../zeroscope_v2_576w", torch_dtype=torch.float16)
    print(type(pipe))
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    # pipe.enable_model_cpu_offload()
    pipe.to("cuda:0")


    for i, a in annots.items():
        if int(i) < start_from:
            continue
        for n in range(n_samples):
            for c in range(len(a['text_descriptions'])):
                # if c==0: continue   # Skip first caption
                prompt = a['text_descriptions'][c]
                generator = torch.Generator(device="cuda").manual_seed(n) # building len(prompts) generators with the exact same seed, so that only the prompt varies
                print("Generating video for prompt:", prompt)
                video_frames = pipe(prompt, 
                                    num_inference_steps=20, 
                                    height=320, 
                                    width=320, 
                                    num_frames=24,
                                    generator=generator).frames # video_frames is a list of frames, not a tensor

                video_name = f'{i}_seed{n}_captionnumber{c}_{prompt.replace("/","-").replace(" ", "-")}'

                # Save frames
                if save_frames:
                    os.makedirs(f'./oracle_gens/frames/{video_name}', exist_ok=True)
                    for k, frame in enumerate(video_frames):
                        plt.imsave(f'./oracle_gens/frames/{video_name}/{(k+1):03d}.png', frame) # We save frames starting with index 1 to match original stimuli

                # Make video and save
                frames_to_vid(video_frames, f'./oracle_gens/mp4/{video_name}.mp4', fps=8)

                # Make gif and save
                vid_to_gif(f'./oracle_gens/mp4/{video_name}.mp4', f'./oracle_gens/gif/{video_name}.gif')


generate_videos_from_annots_with_local(annots, start_from=1001)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

<class 'diffusers.pipelines.text_to_video_synthesis.pipeline_text_to_video_synth.TextToVideoSDPipeline'>
Generating video for prompt: Giant monster truck in an open field with an audience reversing


  0%|          | 0/20 [00:00<?, ?it/s]

[ERROR:0@166.263] global cap_ffmpeg_impl.hpp:3018 open Could not find encoder for codec_id=27, error: Encoder not found
[ERROR:0@166.263] global cap_ffmpeg_impl.hpp:3093 open VIDEOIO/FFMPEG: Failed to initialize VideoWriter


Video written using mp4v codec.
MoviePy - Building file  ./oracle_gens/gif/1001_seed0_captionnumber0_Giant-monster-truck-in-an-open-field-with-an-audience-reversing.gif
MoviePy - - Generating GIF frames.


...

MoviePy - - File ready: ./oracle_gens/gif/1102_seed4_captionnumber3_RC-enthusiast-shows-how-to-assemble-a-new-radio-controlled-truck..gif.
Generating video for prompt: A person is building a powered toy car, which is in an unfinished state.


  0%|          | 0/20 [00:00<?, ?it/s]

[ERROR:0@35497.980] global cap_ffmpeg_impl.hpp:3018 open Could not find encoder for codec_id=27, error: Encoder not found
[ERROR:0@35497.980] global cap_ffmpeg_impl.hpp:3093 open VIDEOIO/FFMPEG: Failed to initialize VideoWriter


Video written using mp4v codec.
MoviePy - Building file  ./oracle_gens/gif/1102_seed4_captionnumber4_A-person-is-building-a-powered-toy-car,-which-is-in-an-unfinished-state..gif
MoviePy - - Generating GIF frames.


MoviePy - - File ready: ./oracle_gens/gif/1102_seed4_captionnumber4_A-person-is-building-a-powered-toy-car,-which-is-in-an-unfinished-state..gif.


In [2]:
# Rename files

# for f in sorted(os.listdir('./oracle_gens/mp4')):
#     # rename {i}_captionnumber{c} to {i}_seed{s}_captionnumber{c}
#     if f[5] == 'c':
#         os.rename(f'./oracle_gens/mp4/{f}', f'./oracle_gens/mp4/{f[:4]}_seed0_{f[5:]}')
#     break


## Convert videos

In [ ]:
from convert_and_move_videos import convert_videos

convert_videos('./oracle_gens/mp4', './oracle_gens/mp4_h264')

In [ ]:
# Check that all videos are there, including their converted and gif versions

def check_videos_exist(path_to_videos= './oracle_gens/', start_idx=1001, end_idx=1102):

    for v in range(start_idx, end_idx+1):
        if int(v[:4]) < start_idx or int(v[:4]) > end_idx:
            continue
        
        for cap in range(5):
            for seed in range(5):
                
                # Check mp4s
                # Check for file that starts with {v}_seed{seed}_captionnumber{cap}
                if not os.path.exists(f'{path_to_videos}/mp4/{v}_seed{seed}_captionnumber{cap}_'):
                    print(f'File {v}_seed{seed}_captionnumber{cap}_ does not exist')
                
                

                




    # Check mp4s_h264s

    # Check gifs

    # Check frames


## Copy videos to NSF

In [ ]:
from convert_and_move_videos import move_to_nsf
    
move_to_nsf('./oracle_gens/mp4_h264', '/data/vision/oliva/scratch/ejosephs/brainGen_eval/stimuli/oracle_gens_zeroscope')
